# Task 2: causal language model

## 1. pkgs loading

In [1]:
import os
# os.environ["WORLD_SIZE"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import torch
import evaluate

## 2. load data

raw dataset can be downloaded from `https://github.com/zyds/transformers-code/tree/master/02-NLP%20Tasks/14-language_model/wiki_cn_filtered`

In [2]:
ds = Dataset.load_from_disk('data/wiki_cn_filtered/')
datasets = ds.train_test_split(test_size=0.1)

In [3]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## 3. data preprocessing

In [6]:
ckpt = 'Langboat/bloom-389m-zh'
tokenizer = AutoTokenizer.from_pretrained(ckpt)

def process_function(examples):
    contents = [e + tokenizer.eos_token for e in examples['completion']]  # Add eos token
    return tokenizer(contents, truncation=True, max_length=384)

tokenized_datasets = datasets.map(process_function, batched=True, num_proc=4, remove_columns=ds.column_names)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [7]:
from torch.utils.data import DataLoader
dl = DataLoader(tokenized_datasets['train'], batch_size=8, shuffle=False, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))

In [8]:
batch = next(enumerate(dl))[1]

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [9]:
batch['labels'][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

In [10]:
batch['input_ids'][0]

tensor([    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3, 

In [11]:
tokenizer.decode(batch['input_ids'][0])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [12]:
tokenizer.eos_token

'</s>'

## 4. create model

In [13]:
model = AutoModelForCausalLM.from_pretrained(ckpt)

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

## 5. config training args

In [20]:
args = TrainingArguments(output_dir='./causal_lm', 
                            per_device_train_batch_size=8,
                            gradient_accumulation_steps=4,
                            per_device_eval_batch_size=8,
                            num_train_epochs=1,
                            logging_steps=10,
                            load_best_model_at_end=True,
                            evaluation_strategy='epoch',
                            save_strategy='epoch',
                            save_total_limit=1,
                            report_to='none',
                            )
trainer = Trainer(model=model, args=args, train_dataset= tokenized_datasets['train'], eval_dataset=tokenized_datasets['test'],
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False))

## 6. train

In [ ]:
trainer.train()

## 7. inference

In [19]:
from transformers import pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的", max_length=64, do_sample=True)

[{'generated_text': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的科研、人文图书馆和中国最大的爱国主义展览馆。馆内藏品由古人类化石、古墓群、出土文物，以及近现代出土的中国革命文物。\n西安交通大学博物馆陈列了来自中国社会各领域的重要"}]